# Working with CSV Files

### Structure of csv files

```csv
column 1 name,column 2 name, column 3 name
first row data 1,first row data 2,first row data 3
second row data 1,second row data 2,second row data 3
...
```

In general the seperator character is called delimiter, the comma is not the only one used.

In [13]:
import requests

data_url = "https://datasets.imdbws.com/name.basics.tsv.gz"
target_csv = "imdb.name.basics.tsv"


def download_dataset(url, target):
    response = requests.get(url)
    response.raise_for_status()
    with open(target + ".gz", "wb") as f:
        f.write(response.content)
    print("Done")


download_dataset(data_url, target_csv)

Done


In [5]:
%%bash

gunzip imdb.name.basics.tsv.gz
head -n 20 imdb.name.basics.tsv > imdb.name.basics.tsv

In [11]:
import csv
from pprint import pprint

with open(target_csv) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter="\t")
    for row in csv_reader:
        print(row[0], row[1])

### Reading csv files to a dictionary

You can directly read the csv to a dict by using `DictReader`

In [35]:
with open(target_csv) as csv_file:
    csv_reader = csv.DictReader(csv_file)
    # for row in csv_reader:
    #     pprint(row)

### Writing csv files with ` csv`

You can write to a CSV file using a writer object and the .write_row() method:

In [14]:
out_csv = "testwrite.csv"
with open(out_csv, mode="w") as out_file:
    csv_writer = csv.writer(
        out_file, delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL
    )

    csv_writer.writerow(["John", "Accounting", "November"])
    csv_writer.writerow(["Eric", "IT", "March"])

In [19]:
!cat testwrite.csv

You can also write data from a dictionary:

In [17]:
out_csv = "testwrite2.csv"
with open(out_csv, mode="w") as out_file:
    fieldnames = ["emp_name", "dept", "birth_month"]
    writer = csv.DictWriter(out_file, fieldnames=fieldnames)

    writer.writeheader()
    writer.writerow(
        {"emp_name": "John", "dept": "Accounting", "birth_month": "November"}
    )
    writer.writerow({"emp_name": "Eric", "dept": "IT", "birth_month": "March"})

In [18]:
!cat testwrite2.csv

## Working with `pandas`

### Reading data

In [24]:
import pandas as pd

df = pd.read_csv("testread.csv")
df.head(len(df))

,Name,Hire Date,Salary,Sick Days remaining
0,Graham Chapman,03/15/14,50000.0,10
1,John Cleese,06/01/15,65000.0,8
2,Eric Idle,05/12/14,45000.0,10
3,Terry Jones,11/01/13,70000.0,3
4,Terry Gilliam,08/12/14,48000.0,7
5,Michael Palin,05/23/13,66000.0,8


You can also use a different column as index:

In [28]:
df = pd.read_csv("testread.csv", index_col="Name")
df.head(len(df))

,Hire Date,Salary,Sick Days remaining
Name,,,
Graham Chapman,03/15/14,50000.0,10
John Cleese,06/01/15,65000.0,8
Eric Idle,05/12/14,45000.0,10
Terry Jones,11/01/13,70000.0,3
Terry Gilliam,08/12/14,48000.0,7
Michael Palin,05/23/13,66000.0,8


Next, let’s fix the data type of the Hire Date field. You can force pandas to read data as a date with the parse_dates optional parameter, which is defined as a list of column names to treat as dates:

In [29]:
df = pd.read_csv("testread.csv", index_col="Name", parse_dates=["Hire Date"])
df.head(len(df))

,Hire Date,Salary,Sick Days remaining
Name,,,
Graham Chapman,2014-03-15,50000.0,10
John Cleese,2015-06-01,65000.0,8
Eric Idle,2014-05-12,45000.0,10
Terry Jones,2013-11-01,70000.0,3
Terry Gilliam,2014-08-12,48000.0,7
Michael Palin,2013-05-23,66000.0,8


If your CSV files doesn’t have column names in the first line, you can use the names optional parameter to provide a list of column names. You can also use this if you want to override the column names provided in the first line. In this case, you must also tell pandas.read_csv() to ignore existing column names using the header=0 optional parameter:

In [35]:
df = pd.read_csv(
    "testread.csv",
    index_col="Employee",
    parse_dates=["Hired"],
    header=0,
    names=["Employee", "Hired", "Salary", "Sick Days"],
)
df.head(len(df))

,Hired,Salary,Sick Days
Employee,,,
Graham Chapman,2014-03-15,50000.0,10
John Cleese,2015-06-01,65000.0,8
Eric Idle,2014-05-12,45000.0,10
Terry Jones,2013-11-01,70000.0,3
Terry Gilliam,2014-08-12,48000.0,7
Michael Palin,2013-05-23,66000.0,8


### Writing data

In [36]:
df = pd.read_csv(
    "testread.csv",
    index_col="Employee",
    parse_dates=["Hired"],
    header=0,
    names=["Employee", "Hired", "Salary", "Sick Days"],
)
df.to_csv("modifiedtestwrite.csv")

In [37]:
!cat modifiedtestwrite.csv

Employee,Hired,Salary,Sick Days
Graham Chapman,2014-03-15,50000.0,10
John Cleese,2015-06-01,65000.0,8
Eric Idle,2014-05-12,45000.0,10
Terry Jones,2013-11-01,70000.0,3
Terry Gilliam,2014-08-12,48000.0,7
Michael Palin,2013-05-23,66000.0,8
